In [12]:
import pandas as pd 
import numpy as np
import json 
import sklearn



In [13]:
df_train_small = pd.read_csv('./data_transformed/train_small_2.csv')


In [14]:
df_train_small

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,hour,day,month
0,11,0,3,790.836,1,449,1414989,45,199.99,-4.8,22,1,24,11
1,9,1,3,4.396,0,305,953684,37,211.31,19.0,22,21,3,7
2,4,0,3,0.000,0,35,103626,16,98.25,8.7,21,5,6,10
3,15,1,1,0.000,0,551,1745132,59,129.23,-4.0,23,19,6,3
4,7,1,1,0.000,0,620,1961232,29,85.59,9.4,23,2,14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,1,0,1004.700,0,298,930074,21,268.00,28.5,22,12,26,6
99996,4,0,3,1.938,1,306,955623,16,150.00,23.8,22,12,4,7
99997,15,1,0,0.000,0,333,1042698,64,471.48,18.0,22,23,31,7
99998,4,1,0,734.972,1,609,1928077,17,101.89,4.0,23,22,3,5


In [15]:
# Drop all lines where target is null or nan
# A function 'dropna' exists also
df_train_small = df_train_small[df_train_small["target"].isna() == False]


# Train - Test 

In [16]:
from sklearn.model_selection import train_test_split

X = df_train_small.drop('target', axis=1)
y = df_train_small['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [24]:
X_train

,county,is_business,product_type,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,hour,day,month
8140,8,0,3,1,636,2014003,32,-1.00,-1.0,23,14,30,5
2008,3,0,3,0,70,211096,12,156.53,9.4,21,18,10,11
89852,13,0,3,0,18,54026,51,172.67,8.4,21,10,19,9
88267,0,0,2,0,133,402060,1,134.13,-8.9,22,13,12,1
76307,10,1,1,0,501,1584572,40,115.97,5.0,23,14,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6267,7,0,3,0,542,1714974,27,99.94,-1.7,23,8,25,2
54899,9,1,1,1,545,1725945,36,183.69,0.3,23,18,28,2
76840,4,0,3,0,197,603554,16,284.51,-5.8,22,7,17,3
860,3,1,1,0,117,351840,13,149.96,1.1,21,7,27,12


In [18]:
y_train.isna().sum()

0

In [19]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)

DummyRegressor()

In [20]:
from sklearn.linear_model import LinearRegression
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

reg = LinearRegression().fit(X_train, y_train)
y_train_hat = reg.predict(X_train)

In [25]:
# recherche des meilleurs paramètres
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

parameters = {'alpha': [0.01, 0.1, 1, 10, 100]}

lasso = Lasso()

grid_search = GridSearchCV(lasso, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

BestAlpha = grid_search.best_params_['alpha']

# entraînement du modèle
BestLasso = Lasso(alpha=BestAlpha)
BestLasso.fit(X_train, y_train)



C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.376e+10, tolerance: 5.387e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.385e+10, tolerance: 5.431e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

Lasso(alpha=0.1)

In [27]:
#  meilleur modèle pour prédire le premier dataset
lasso_res = BestLasso.predict(X_train)

# marge d'erreur du modèle
lasso = abs(lasso_res - y_train)


In [31]:
from sklearn.ensemble import RandomForestRegressor

# création d'une instance du modèle RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)  # Vous pouvez ajuster les hyperparamètres selon vos besoins

# Entraînement du modèle sur les données d'entraînement
rf_reg.fit(X_train, y_train)

#prédictions sur les données d'entraînement
y_train_hat_rf = rf_reg.predict(X_train)




In [33]:
import xgboost as xgb

# Création d'une instance du modèle XGBoost pour la régression
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# la grille des hyperparamètres à rechercher
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2 ,0.3 ,0.5 ,1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
}

# GridSearchCV pour rechercher les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres:", grid_search.best_params_)

# prédictions sur les données d'entraînement avec les meilleurs paramètres
y_train_hat_xgb = grid_search.predict(X_train)



Meilleurs paramètres: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300}


In [40]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Supposons que X_train, y_train soient vos données d'entraînement
# et X_test, y_test soient vos données de test

# Créez une instance du modèle XGBoost pour la régression
xgb_reg2 = xgb.XGBRegressor(
    objective='reg:squarederror',  # Pour la régression
    learning_rate=0.2,             # Taux d'apprentissage
    n_estimators=300,              # Nombre d'estimateurs (arbres)
    max_depth=5,                   # Profondeur maximale de l'arbre
    subsample=0.8,                 # Sous-échantillonnage des lignes
    colsample_bytree=0.8,          # Sous-échantillonnage des colonnes
    gamma=0,                       # Paramètre de régularisation
    reg_lambda=1,                  # Régularisation L2
    reg_alpha=0,                   # Régularisation L1
    min_child_weight=1             # Poids minimal des feuilles
)

# Entraînez le modèle sur les données d'entraînement
xgb_reg.fit(X_train, y_train)

# Faites des prédictions sur les données de test
y_test_hat_xgb = xgb_reg.predict(X_train)



ValueError: Found input variables with inconsistent numbers of samples: [19995, 79980]

## Metrics and Evaluation

In [38]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

def model_evaluator(model_trained):
  temp_y_train_hat = model_trained.predict(X_train)
  print("MAE : ", mean_absolute_error(y_train, temp_y_train_hat))
  print("MSE : ", mean_squared_error(y_train, temp_y_train_hat))
  print("MAPE : ", mean_absolute_percentage_error(y_train, temp_y_train_hat))

In [22]:
model_evaluator(dummy_regr)

MAE :  371.60640319877416
MSE :  834904.0591048338
MAPE :  2.1997051336536547e+17


In [23]:
model_evaluator(reg)

MAE :  368.1120715674413
MSE :  735873.4275985473
MAPE :  1.699930442636103e+17


In [30]:
model_evaluator(BestLasso)

MAE :  367.94886205299076
MSE :  735898.4473060821
MAPE :  1.6986606811659267e+17


In [32]:
model_evaluator(rf_reg)

MAE :  25.775425765316335
MSE :  8210.614735726971
MAPE :  472435991526086.75


In [39]:
model_evaluator(xgb_reg2)

NotFittedError: need to call fit or load_model beforehand